In [ ]:
import tensorflow as tf
import numpy as np
import pylab as plt
from regular_training.classifier_utils import load_from_directory
from nn_robust_attacks.l2_attack import CarliniL2

from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense
from keras.layers import Input
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
%matplotlib inline

In [ ]:
IMG_SIZE = 48
NUM_CLASSES = 10

Model definition without Softmax layer (requirement by Carlini-Wagner attack):

In [ ]:
class traffic_signs_classifier:
    def __init__(self, weights, session=None):
        self.num_channels = 3
        self.image_size = IMG_SIZE
        self.num_labels = NUM_CLASSES
        
        model = Sequential()

        model.add(Conv2D(32, (3, 3), padding='same', 
                            input_shape=(48, 48, 3),
                            activation='relu'))
        model.add(Conv2D(32, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Conv2D(64, (3, 3), padding='same',
                            activation='relu'))
        model.add(Conv2D(64, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))

        model.add(Conv2D(128, (3, 3), padding='same', 
                            activation='relu'))
        model.add(Conv2D(128, (3, 3), activation='relu'))
        model.add(MaxPooling2D(pool_size=(2, 2)))
        model.add(Flatten())
        model.add(Dense(512, activation='relu'))
        model.add(Dense(NUM_CLASSES))

        model.load_weights(weights)
        self.model = model
        
    def predict(self, data):
        return self.model(data)

In [ ]:
images,labels = load_from_directory("./examples", IMG_SIZE, NUM_CLASSES)

In [ ]:
with tf.Session() as sess:
    
    model = traffic_signs_classifier("./regular_training/regular_training.h5",sess)
    images,labels = load_from_directory("./examples", IMG_SIZE, NUM_CLASSES)
    
    # setting target labels to the label #9
    target_labels = np.zeros((len(labels), 10), dtype=np.int)
    for i in range(len(target_labels)): target_labels[i][9] = 1
    
    attack = CarliniL2(sess, model, confidence = 20)
    adversarial_examples = attack.attack(images-0.5,target_labels)
    
    # saving adversarial images to files
    for i in range(len(adversarial_examples)):
        path = "./adversarial_examples/" + "adv_image_" + str(i) + ".png"
        plt.imsave(path,adversarial_examples[i]+0.5)